# Observational study 

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from networkx import nx
from plotnine import ggplot, aes, geom_bar
from functions import *
import spacy
import time
import datetime
import statsmodels.formula.api as smf

In [3]:
from popularity import trending_history
from financials import *

In [4]:
elon_org_df = pd.read_csv('Data/org-lg-Elon Musk.csv.bz2') 94 64 62 20

In [47]:
e = elon_org_df[elon_org_df['ORG']== 'Apple']
f = e['date']
print (f.head (10))
for i in range (len(f)):
    f.iloc[i] = f.iloc[i][0:10]
f.unique().shape    

7      2015-10-09 11:30:14
18     2015-10-09 16:50:01
24     2015-05-06 23:48:00
29     2015-12-26 06:45:28
37     2015-12-14 22:52:19
41     2015-10-10 06:28:35
45     2015-10-09 15:15:00
63     2015-10-09 06:30:37
92     2015-10-09 15:03:42
105    2015-05-09 23:58:42
Name: date, dtype: object


(44,)

In [43]:
f

7       2015-10-09
18      2015-10-09
24      2015-05-06
29      2015-12-26
37      2015-12-14
           ...    
3958    2018-11-03
4242    2018-11-02
5082    2019-07-23
5125    2019-01-08
5771    2020-03-12
Name: date, Length: 94, dtype: object

## Matching : 

In [24]:
data = pd.read_csv('Data/FinalFeatures.csv.bz2')
a= data[data['Date']=='2015-10-09']
a[a['Name']== 'Apple']

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare
4,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
9,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
34,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
39,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
44,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
89,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
94,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
104,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
109,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
114,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None


In [ ]:
#Propensity score : 
def add_propensityscore (data):#so that if we want to change only the first line don't have to do it
    #With a column of one and zero if elon musk talked about you 
    #company = company_quote.extend(controled)
    #data = compareCompanies(company,date)
    #Normalise the features 
    data['Close'] = (data['Close'] - data['Close'].mean())/data['Close'].std()
    data['Volume']= (data['Volume']-data['Volume'].mean())/data['Volume'].std()
    data['MarketCap']= (data['MarketCap']-data['MarketCap'].mean())/data['MarketCap'].std()
    data['Popularity']= (data['Popularity']-data['Popularity'].mean())/data['Popularity'].std()
    #Create the model
    mod = smf.logit(formula='Elon ~ Close+ Volume + MarketCap + Popularity', data=data)
    res = mod.fit()
    data['Propensity_score'] = res.predict()

In [ ]:
add_propensityscore(data)

In [ ]:
data[data['Name']== 'Samsung']

In [ ]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [ ]:
def matchingfunction(companies):
    treatment_df = data[data['Elon'] == 1]
    control_df   = data[data['Elon'] == 0]
    # Create an empty undirected graph
    G = nx.Graph()
    # Loop through all the pairs of instances
    for control_id, control_row in control_df.iterrows():
        for treatment_id, treatment_row in treatment_df.iterrows():
            if (control_row['compare']== treatment_row['Name']) and (control_row['Date']== treatment_row['Date']) :
                # Calculate the similarity 
                similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])
                # Add an edge between the two instances weighted by the similarity between them
                G.add_weighted_edges_from([(control_id, treatment_id, similarity)])
    # Generate and return the maximum weight matching on the generated graph
    matching = nx.max_weight_matching(G)
    return matching

In [ ]:
matching = matchingfunction(data)
matched = [i[0] for i in list(matching)]+ [i[1]for i in list(matching)]
balanced = data.iloc[matched]


## After matching : 


In [ ]:
data[data['Name']== 'Apple']

In [ ]:
matching (data)

In [ ]:
treated = balanced.loc[balanced['Elon']==1]

In [ ]:
controled = balanced.loc[balanced['Elon']==0]

In [ ]:
treated

In [ ]:
controled 

In [ ]:
matching


In [ ]:
matched

In [ ]:
data_Apple[data_Apple['Date']=='2015-10-09']

In [ ]:
before =time.time()
quotes_Apple = elon_org_df[elon_org_df['ORG'] == 'Apple']
first_date = quotes_Apple.iloc[0]['date'][0:10]
Company_Apple =['Microsoft','IBM','Samsung','Dell','Apple'] 
ticker_Apple =[ticker_of_company('Microsoft'),
               ticker_of_company('IBM'),
               ticker_of_company('Samsung'),
               ticker_of_company('Dell'),
              ticker_of_company('Apple')] 
shares_Apple =[yf.Ticker(ticker_Apple[0]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[1]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[2]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[3]).info['sharesOutstanding'],
               yf.Ticker(ticker_Apple[4]).info['sharesOutstanding']]
hist_Apple =[yf.Ticker(ticker_Apple[0]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[1]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[2]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[3]).history(start=first_date)[['Close', 'Volume']],
               yf.Ticker(ticker_Apple[4]).history(start=first_date)[['Close', 'Volume']]]
after =time.time()
print('fetch time:',str(after-before))

In [ ]:
hist_Apple[0].iloc[0]

In [ ]:
hist_Apple[0].iloc[0].name

In [ ]:
hist_Apple[0].iloc[0].name.date() > datetime.date.fromisoformat('2015-12-04')

In [ ]:
for i in range(len(hist_Apple[0])):
    if hist_Apple[0].iloc[i].name.date() > datetime.date.fromisoformat('2015-12-12'):
        print(hist_Apple[0].iloc[i])
        break